<a href="https://colab.research.google.com/github/SeanBoyd13/BIG_DATA_CUP_2025_Quantifying_Off_Puck_Positioning/blob/main/code/colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files

uploaded = files.upload()

file_name = list(uploaded.keys())[0]

df = pd.read_csv(file_name)

Saving 2024-11-16.Team.F.@.Team.E.-.Tracking.csv to 2024-11-16.Team.F.@.Team.E.-.Tracking (2).csv


<ipython-input-20-f6d5feed15ee>:8: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)


In [ ]:
def load_game(df):
    game_frames = []
    frame = []
    current_frame_id = None

    for _, row in df.iterrows():
        object_type = row['Player or Puck']
        team = row['Team']
        player_id = row['Player Id']
        x = row['Rink Location X (Feet)']
        y = row['Rink Location Y (Feet)']
        z = row['Rink Location Z (Feet)']
        frame_id = row['Image Id']

        if frame_id != current_frame_id:
            if frame:
                game_frames.append(frame)
            frame = []
            current_frame_id = frame_id

        if pd.isna(object_type):
            continue
        if object_type == "Player" and pd.isna(player_id):
            continue

        try:
            if object_type == "Puck":
                entry = [object_type, "Puck", team, float(x), float(y)]
            else:
                entry = [object_type, player_id, team, float(x), float(y)]
        except ValueError:
            continue

        frame.append(entry)

    if frame:
        game_frames.append(frame)

    print(f"Total frames: {len(game_frames)}")
    return game_frames

tracking_data = load_game(df)

Total frames: 105557


In [ ]:
# Index:   0                1    2             3  4
# Entry: ["Player"/"Puck", "#", "Home"/"Away", x, y]

players_controlled_xG_O = {}
players_controlled_xG_D = {}
current_goalie_x = {}

last_frame_player_locations = {}
second_last_frame_player_locations = {}

MAX_SUM = 337.6415463329972

def load_data_from_frame(frame):
  # Extracts player and puck locations from the frame
  player_locations = {}

  for entry in frame:
    if entry[0] == "Player":
      object_type, player_id, team, x, y = entry

      if player_id == "Go":
        current_goalie_x[team] = x # Tracks goalie location for side identification
        continue

      player_id = str(str(team)+", #"+str(player_id))

      player_locations[player_id] = [x,y]

    else:
      puck_location = [entry[3], entry[4]]

  return player_locations, puck_location


def distance(point, target):
  # Calculates Euclidean distance between two points
  point_x = int(point[0])
  point_y = int(point[1])

  target_x = target[0]
  target_y = target[1]

  return ((point_x - target_x)**2 + (point_y - target_y)**2)**0.5


def closest_player(point, players):
  # Determines closest player by Euclidean distance from point
  min_distance = 10000
  closest_player = None

  for player in players:
    dist = distance([(players[player])[0], (players[player])[1]], [point[0], point[1]])

    if dist < min_distance:
      min_distance = dist
      closest_player = player

  return closest_player


def get_velocity(position, last_frame_position):
  # Calculates velocity using previous location

  dt = 1

  vx = (position[0] - last_frame_position[0]) / dt
  vy = (position[1] - last_frame_position[1]) / dt

  velocity_vector = [vx, vy]

  return velocity_vector


def get_acceleration(player, player_locations):
  # Calculates acceleration using past locations

  player_location = player_locations[player]

  position = player_location
  last_frame_position = last_frame_player_locations.get(player)
  second_last_position = second_last_frame_player_locations.get(player)

  if last_frame_position is None or second_last_position is None:
    return None

  velocity_vector1 = get_velocity(position, last_frame_position)
  velocity_vector2 = get_velocity(last_frame_position, second_last_position)

  if velocity_vector1 is None or velocity_vector2 is None:
    return None

  dt = 1

  ax = (velocity_vector1[0] - velocity_vector2[0]) / dt
  ay = (velocity_vector1[1] - velocity_vector2[1]) / dt

  acceleration_vector = [ax, ay]

  return acceleration_vector


def time_to_point(point, player, player_locations):
  # Calculates time to reach a point given location, velocity, and acceleration
  # Assumes most recent magnitude of acceleration, now in direct of point

  player_location = player_locations[player]

  if player not in last_frame_player_locations.keys() or player not in second_last_frame_player_locations.keys():
    return 100000

  position = player_location
  velocity_vector = get_velocity(position, last_frame_player_locations.get(player))
  acceleration_vector = get_acceleration(player, player_locations)

  if velocity_vector is None or acceleration_vector is None:
    return 100000

  dx = point[0] - position[0]
  dy = point[1] - position[1]

  distance_to_point = distance(position, point)

  target_direction =[dx / distance_to_point, dy / distance_to_point]

  vx, vy = velocity_vector
  ax, ay = acceleration_vector

  acceleration_magnitude = (ax**2 + ay**2)**0.5

  vel_towards_target = vx * target_direction[0] + vy * target_direction[1]
  acc_towards_target = acceleration_magnitude * target_direction[0] + acceleration_magnitude * target_direction[1]

  if acc_towards_target == 0:
    if vel_towards_target == 0:
      return 100000
    else:
      return distance_to_point / vel_towards_target
  else:
    discriminant = vel_towards_target**2 + 2 * distance_to_point * acc_towards_target

    if discriminant < 0:
      return 100000
    else:
      return (-vel_towards_target + discriminant**0.5) / acc_towards_target


def fastest_to_point(point, player_locations):
  # Determines player who will reach a point the fastest
  min_time = 10000
  fastest_player = None

  for player in players:
    time = time_to_point(point, player, player_locations)

    if time < min_time:
      min_time = time
      fastest_player = player

  # Update previous locations for next frame
  global second_last_frame_player_locations, last_frame_player_locations
  second_last_frame_player_locations = last_frame_player_locations.copy()
  last_frame_player_locations = player_locations.copy()

  if fastest_player is None: # If location in previous frame is inaccessible, distance is used instead
    return closest_player(point, player_locations)

  return fastest_player


def get_zone(puck):
  # Determines x-coordinate bounds for offensive/defensive zone
  if puck[0] >= 25:
    zone = [25, 100]
  elif puck[0] <= -25:
    zone = [-100, -25]
  else:
    zone = None

  return zone


def xG(point):
  # Calculates rough estimation of expected goals using distance from goal
  x = point[0]
  y = point[1]

  if x >= 25:
    dist_from_goal = distance(point, [89, 0])
  else:
    dist_from_goal = distance(point, [-89, 0])

  xG = 1 - (dist_from_goal / (dist_from_goal + 1.5))

  return xG


def evaluate_frame(players, puck):
  # Calculates closest player to each point within zone and calculates xG
  # Produces controlled xG; the sum of all controlled points' xG out of the maximum possible sum

  try:
    zone_bounds = get_zone(puck)

    if zone_bounds is None:
      return

    x_bounds = zone_bounds
    y_bounds = [-42.5, 42.5]

    controlled_points = {}

    for i in range(75):
      for j in range(85):
        x = x_bounds[0] + i
        y = y_bounds[0] + j

        point = [x, y]

        closest = fastest_to_point(point, players)

        if closest not in controlled_points:
          controlled_points[closest] = []

        controlled_points[closest].append(point)

    for player in controlled_points.keys():
      points = controlled_points[player]

      player_total_xG = 0

      if closest_player(puck, players) == player and distance(players[player], puck) <= 5 : # Off-puck players only
        continue

      for point in points:
        player_total_xG += (xG(point) / MAX_SUM)

      p_array = player.split(",")
      team = p_array[0]

      goalie_x = current_goalie_x[team]
      player_location = players[player]
      player_x = player_location[0]

      if (goalie_x >= 0 and player_x >= 0) or (goalie_x < 0 and player_x < 0):

        if player not in players_controlled_xG_D:
          players_controlled_xG_D[player] = []

        players_controlled_xG_D[player].append(round(player_total_xG,2))
        continue

      elif (goalie_x < 0 and player_x >= 0) or (goalie_x >= 0 and player_x < 0):

        if player not in players_controlled_xG_O:
          players_controlled_xG_O[player] = []

        players_controlled_xG_O[player].append(round(player_total_xG,2))
        continue

      else:
        continue

    return

  except:
    return

for i, frame in enumerate(tracking_data):
  # Every 30th frame is used for efficiency, since increased specificity yields minimal difference in results
  if i % 30 != 0:
      continue
  players, puck = load_data_from_frame(frame)
  evaluate_frame(players, puck)

print(players_controlled_xG_O)
print(players_controlled_xG_D)

{'Home, #68': [0.01, 0.04, 0.06, 0.15, 0.16, 0.12, 0.14, 0.1, 0.11, 0.09, 0.06, 0.02, 0.05, 0.04, 0.06, 0.21, 0.26, 0.25, 0.25, 0.16, 0.03, 0.05, 0.05, 0.04, 0.05, 0.07, 0.1, 0.22, 0.08, 0.04, 0.01, 0.02, 0.01, 0.01, 0.01, 0.36, 0.25, 0.06, 0.11, 0.14, 0.11, 0.15, 0.2, 0.22, 0.16, 0.29, 0.58, 0.21, 0.07, 0.01, 0.03, 0.16, 0.11, 0.13, 0.09, 0.07, 0.1, 0.27, 0.01, 0.15, 0.13, 0.22, 0.11, 0.03, 0.01, 0.0, 0.03, 0.11, 0.27, 0.36, 0.19, 0.09, 0.02, 0.04, 0.06, 0.05, 0.97, 0.06, 0.18, 0.04, 0.01, 0.03, 0.04, 0.02, 0.01, 0.03, 0.12, 0.04, 0.06, 0.2, 0.23, 0.1, 0.01, 0.01, 0.02, 0.02, 0.03, 0.03, 0.36, 0.29, 0.3, 0.22, 0.12, 0.03, 0.01, 0.01, 0.09, 0.09, 0.06, 0.04, 0.04, 0.2, 0.41, 0.4, 0.28, 0.12, 0.18, 0.03, 0.01, 0.03, 0.18, 0.3, 0.29, 0.4, 0.47, 0.49, 0.52, 0.46, 0.43, 0.24, 0.36, 0.32, 0.23, 0.21, 0.08, 0.06, 0.1, 0.28, 0.21, 0.25, 0.2, 0.22, 0.03, 0.03, 0.01, 0.1, 0.03, 0.04, 0.02, 0.19, 0.27, 0.32, 0.32, 0.02, 0.02, 0.02, 0.04, 0.21, 0.47, 0.29, 0.21, 0.24, 0.24, 0.13, 0.0, 0.11, 0.21,

In [ ]:
print("OFFENSIVE ZONE:")
print("Team, Number, Control, Significant Control")
for player in players_controlled_xG_O.keys():
  data = players_controlled_xG_O[player]

  sum = 0
  count = 0
  significant = 0
  for datapoint in data:
    count += 1
    sum += datapoint

    if datapoint >= .15: # Players controlling 50% greater ice than expected (expected 10% for all 10 players on ice)
      significant += 1

  try:
    print(f"{player}, {round(sum/count,3)}, {significant}")
  except:
    print(player, "DIV/0")
    continue

print("DEFENSIVE ZONE:")
print("Team, Number, Control, Significant Control")
for player in players_controlled_xG_D.keys():
  data = players_controlled_xG_D[player]

  sum = 0
  count = 0
  significant = 0
  for datapoint in data:
    count += 1
    sum += datapoint

    if datapoint >= .15:
      significant += 1

  try:
    print(f"{player}, {round(sum/count,3)}, {significant}")
  except:
    print(player, "DIV/0")
    continue

OFFENSIVE ZONE:
Team, Number, Control, Significant Control
Home, #68, 0.136, 116
Home, #46, 0.106, 98
Home, #19, 0.086, 71
Home, #29, 0.092, 47
Home, #4, 0.078, 24
Away, #29, 0.122, 106
Away, #91, 0.107, 91
Home, #11, 0.109, 82
Home, #47, 0.08, 28
Home, #12, 0.105, 83
Home, #18, 0.085, 20
Away, #13, 0.109, 83
Home, #36, 0.129, 107
Home, #42, 0.1, 62
Away, #38, 0.138, 62
Away, #71, 0.099, 40
Away, #22, 0.118, 55
Home, #25, 0.087, 49
Home, #45, 0.068, 19
Away, #64, 0.073, 15
Home, #77, 0.092, 42
Away, #26, 0.102, 41
Away, #49, 0.104, 26
Away, #36, 0.105, 49
Away, #78, 0.1, 19
Away, #5, 0.071, 21
Home, #89, 0.104, 42
Home, #75, 0.112, 73
Home, #20, 0.077, 21
Away, #27, 0.1, 41
Away, #56, 0.101, 53
Away, #21, 0.09, 21
Away, #3, 0.093, 16
Away, #55, 0.069, 11
Away, #42, 0.113, 34
Home, #44, 0.049, 1
DEFENSIVE ZONE:
Team, Number, Control, Significant Control
Home, #4, 0.226, 158
Home, #29, 0.248, 198
Home, #68, 0.094, 38
Away, #55, 0.216, 233
Away, #27, 0.122, 128
Away, #5, 0.231, 268
Away, 